In [25]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import sys, traceback
import json

In [2]:
KW_CODE =0
KM_CODE =1
DJ_CODE=2
DS_CODE=3
DD_CODE=4
MJ_CODE=5
SY_CODE=6
SM_CODE=7
SK_CODE=8
SW_CODE=9
SS_CODE=10
HS_CODE=11

In [3]:
base_start_url_list = ['http://kupis.kw.ac.kr/',
'http://lib.kookmin.ac.kr/',
'http://library.daejin.ac.kr/',
'http://discover.duksung.ac.kr/',
'http://library.dongduk.ac.kr/',
'http://lib.mju.ac.kr/',
'http://lib.syu.ac.kr/',
'https://lib.smu.ac.kr/',
'http://library.skuniv.ac.kr/',
'http://lib.swu.ac.kr/',
'http://lib.sungshin.ac.kr/',
'http://hsel.hansung.ac.kr/']


In [4]:
base_middle_url_list = ['',
'',
'',
'',
'',
'',
'',
'Search/?q=',
'',
'',
'',
'']

In [5]:
base_end_url_list = []
for i in range(12):
    base_end_url_list.append('')

In [6]:
class LibraryLS:
    def __init__(self, sid):
        self.sid = sid
        self.loanStatusList = []
        self.errorMessage= ''
    

In [40]:
class LoanStatus:
    def __init__(self, title = '', RN = '', CN = '', POS = '', STATE = -1, RDD = '', BN = 0, errorMessage = ''):
        self.title = title
        self.RN= RN
        self.CN= CN
        self.POS= POS
        self.STATE= STATE
        self.RDD = RDD#'2222-12-31'
        self.BN = BN
        self.errorMessage = errorMessage
'''#STATE : 도서 상태,
#-1  에러
#1 대출가능
#0 대출중
#2 지정도서
# 대출불가 아직 생각 안함.
#RN : 등록번호
#CN : 청구기호
#POS : 위치
이런것들'''

'#STATE : 도서 상태,\n#-1  에러\n#1 대출가능\n#0 대출중\n#2 지정도서\n# 대출불가 아직 생각 안함.\n#RN : 등록번호\n#CN : 청구기호\n#POS : 위치\n이런것들'

In [43]:
#아직 상명대만 함. 상명대 ISBN으로 검색예정.
def makeSearchUrl(ISBN, title, sid, flag):
    if flag ==1:
        searchUrl = base_start_url_list[sid]+base_middle_url_list[sid] + ISBN+ base_end_url_list[sid]
        return searchUrl
    

In [44]:
def findSubStringByRegEx(text, sid, subStringType):
    regex = re.compile(regexDictList[sid][subStringType])
    matchObj = regex.search(text)
    subString = matchObj.group(1)
    return subString
    
    

In [45]:
def find_ISBN(text):
    regex = re.compile('\d\d\d\d\d\d\d\d\d\d\d\d\d')
    matchObj = regex.search(text)
    return matchObj.group(0)

In [46]:
def find_BN(text):
    if '예약' in text:
        regex = re.compile('대출중\s*\(\s+(\d+)명')
        matchObj = regex.search(text)
        num = int(matchObj.group(1))
        return num
    else :
        return 0

In [47]:
def find_RDD(text):
    regex = re.compile('\d\d\d\d\-\d\d\-\d\d')
    matchObj = regex.search(text)
    return matchObj.group(0)

In [48]:
def find_STATE_BN(text):
    if '대출중' in text:
        return 0, find_BN(text)
    elif '이용가능' in text:
        return 1,0
    elif '지정도서' in text:
        return 2,0
    else :
        return -1,0

In [58]:
def visitLink(ISBN, sid, title, bookLink):
    loanStatusList = []
    #= LoanStatus()
    try:
        #1. 접속.
        html = urlopen(base_start_url_list[sid]+bookLink)
        bsObject = BeautifulSoup(html, "html.parser")
        
        #1-1 ISBN 정보가 일치하지 않다면 바로 return!!!
        includedISBNString = str(bsObject.find('div', {'class':'col-md-10 detail-table-right'}).find_all('dl')[2].find('dd'))

        if ISBN != find_ISBN(includedISBNString):
            return loanStatusList
        #2. 대출 데이터를 차곡차곡저장.
        idx = 0
        for loanResult in bsObject.find('div', class_='sponge-guide-Box-table sponge-detail-table').find('tbody').find_all('tr'):
            errorMessage = ''
            content = loanResult.find_all('td')
            RN = content[0].text.strip()
            CN = content[1].text.strip()
            POS = content[2].text.strip()
            STATE_STRING = content[3].text.strip()
            try:
                RDD = find_RDD(str(content[4]))
            except AttributeError as e:
                RDD = ''
            STATE, BN = find_STATE_BN(STATE_STRING)
            loanStatusList.append(LoanStatus(title, RN, CN,POS,STATE, RDD ,BN, ''))
    except :
        pass
        #traceback.format_exc()
    finally:
        return loanStatusList

In [59]:
test_string = ""
def crawling(ISBN, title, sid, searchUrl):
    libraryLoanStatus = LibraryLS(sid)
    try:
        #1. 접속.
        html = urlopen(searchUrl)
        bsObject = BeautifulSoup(html, "html.parser")
        #2. 링크를 차곡차곡저장.
        # 정규식과 beautifulsoap4을 이용하여 저장...!!
        bookLinklist =[]
        for bookResult in bsObject.find_all('div', class_='search-list-result'):
            content = bookResult.find('div', {'class','sponge-list-content'}).find('a')
            title = content.text.strip()
            link =  content.get('href')
            bookLinklist.append([title, link])

        #3. 해당 링크를 방문하여 리스트에 추가해라..!!
        for item in bookLinklist:
            libraryLoanStatus.loanStatusList += visitLink(ISBN, sid, item[0], item[1])
    except :
        libraryLoanStatus.errorMessage = traceback.format_exc()
        #libraryLoanStatus.errorMessage = 에러 traceback 에러메세지를 담는다.
    finally:
        return libraryLoanStatus
    

In [41]:
def makeJsonDict(result):
    # result 의 타입은 LibraryLS
    jsonDict = {}
    jsonDict['sid'] = result.sid
    jsonDict['loanStatusList'] = []
    for item in result.loanStatusList:
        jsonDict['loanStatusList'].append(vars(item))
    jsonDict['errorMessage']=result.errorMessage
    return jsonDict

In [42]:
#flag =1 iSBN, 0, title search by..
#함수 만들기.

#sid = int(sid)
def findLoanStatus(ISBN, title, sid):
    searchUrl = makeSearchUrl(ISBN, title, sid, 1)
    result = crawling(ISBN, title, sid, searchUrl)
    # json string 으로 만들기.
    jsonDict = makeJsonDict(result)
    return json.dumps(jsonDict)
    

# 밑에는 예시

In [60]:
ISBN = '9788988474839'
title = '데이터 분석 전문가 가이드'
sid = SM_CODE
#searchUrl = makeSearchUrl(ISBN, title, sid, 1)
jsonString = findLoanStatus(ISBN, title, sid)

In [61]:
print(result)

{"sid": 7, "loanStatusList": [{"title": "\ub370\uc774\ud130 \ubd84\uc11d \uc804\ubb38\uac00 \uac00\uc774\ub4dc  = The guide for advanced data analytics professional", "RN": "L0113396", "CN": "005.74 \ub370\uc774\ud55c\uac1c", "POS": "\uc81c1\uc790\ub8cc\uc2e4(3\uce35)", "STATE": 0, "RDD": "2021-03-01", "BN": 1, "errorMessage": ""}, {"title": "\ub370\uc774\ud130 \ubd84\uc11d \uc804\ubb38\uac00 \uac00\uc774\ub4dc", "RN": "L0112135", "CN": "005.74 \ud55c\uad6d\ub370\uac1c  c.3", "POS": "\uc81c1\uc790\ub8cc\uc2e4(3\uce35)", "STATE": 0, "RDD": "2020-04-02", "BN": 0, "errorMessage": ""}, {"title": "\ub370\uc774\ud130 \ubd84\uc11d \uc804\ubb38\uac00 \uac00\uc774\ub4dc", "RN": "S0480361", "CN": "005.74 \ud55c\uad6d\ub370\uac1c", "POS": "\uc81c1\uc790\ub8cc\uc2e4(3\uce35)", "STATE": 1, "RDD": "", "BN": 0, "errorMessage": ""}, {"title": "\ub370\uc774\ud130 \ubd84\uc11d \uc804\ubb38\uac00 \uac00\uc774\ub4dc", "RN": "S0484218", "CN": "005.74 \ud55c\uad6d\ub370\uac1c  2", "POS": "\uc81c1\uc790\ub8c

In [34]:
jsonString = findLoanStatus('9788970506432', 'idonknow', SM_CODE)

In [35]:
jsonString

'{"sid": 7, "loanStatusList": [{"title": "(C\\uc5b8\\uc5b4\\ub85c \\uc27d\\uac8c \\ud480\\uc5b4 \\uc4f4) \\uc790\\ub8cc \\uad6c\\uc870 = Data structures in C", "RN": "S0399964", "CN": "005.73 \\ucc9c\\uc778\\uc790\\uac1c", "POS": "\\uc81c1\\uc790\\ub8cc\\uc2e4(\\uacfc\\uc81c\\ub3c4\\uc11c)", "STATE": 2, "RDD": "", "BN": 0, "errorMessage": ""}, {"title": "(C\\uc5b8\\uc5b4\\ub85c \\uc27d\\uac8c \\ud480\\uc5b4 \\uc4f4) \\uc790\\ub8cc \\uad6c\\uc870 = Data structures in C", "RN": "S0445167", "CN": "005.73 \\ucc9c\\uc778\\uc790\\uac1c  2", "POS": "\\uc81c1\\uc790\\ub8cc\\uc2e4(\\uacfc\\uc81c\\ub3c4\\uc11c)", "STATE": 2, "RDD": "", "BN": 0, "errorMessage": ""}, {"title": "(C\\uc5b8\\uc5b4\\ub85c \\uc27d\\uac8c \\ud480\\uc5b4 \\uc4f4) \\uc790\\ub8cc \\uad6c\\uc870 = Data structures in C", "RN": "S0445743", "CN": "005.73 \\ucc9c\\uc778\\uc790\\uac1c  3", "POS": "\\uc81c1\\uc790\\ub8cc\\uc2e4(\\uacfc\\uc81c\\ub3c4\\uc11c)", "STATE": 2, "RDD": "", "BN": 0, "errorMessage": ""}, {"title": "(C\\uc5

In [36]:
ilovehangul = json.loads(jsonString)

In [37]:
ilovehangul

{'sid': 7,
 'loanStatusList': [{'title': '(C언어로 쉽게 풀어 쓴) 자료 구조 = Data structures in C',
   'RN': 'S0399964',
   'CN': '005.73 천인자개',
   'POS': '제1자료실(과제도서)',
   'STATE': 2,
   'RDD': '',
   'BN': 0,
   'errorMessage': ''},
  {'title': '(C언어로 쉽게 풀어 쓴) 자료 구조 = Data structures in C',
   'RN': 'S0445167',
   'CN': '005.73 천인자개  2',
   'POS': '제1자료실(과제도서)',
   'STATE': 2,
   'RDD': '',
   'BN': 0,
   'errorMessage': ''},
  {'title': '(C언어로 쉽게 풀어 쓴) 자료 구조 = Data structures in C',
   'RN': 'S0445743',
   'CN': '005.73 천인자개  3',
   'POS': '제1자료실(과제도서)',
   'STATE': 2,
   'RDD': '',
   'BN': 0,
   'errorMessage': ''},
  {'title': '(C언어로 쉽게 풀어 쓴) 자료 구조 = Data structures in C',
   'RN': 'S0445744',
   'CN': '005.73 천인자개  4',
   'POS': '제1자료실(과제도서)',
   'STATE': 2,
   'RDD': '',
   'BN': 0,
   'errorMessage': ''}],
 'errorMessage': ''}

In [167]:
try:
    1/0
    
except AttributeError as e :
    print('AE')
except ZeroDivisionError as e:
    data = str(traceback.format_exc())
finally :
    'fuck'

In [168]:
data

'Traceback (most recent call last):\n  File "<ipython-input-167-bb8ad90923a1>", line 2, in <module>\n    1/0\nZeroDivisionError: division by zero\n'

In [125]:
result = find_RDD(text)

In [126]:
result 

'2021-03-01'

In [90]:
li = [1,2,3,4]
li2 = [4,5,6]

In [91]:
li += li2

In [92]:
li

[1, 2, 3, 4, 4, 5, 6]

In [170]:
print(vars(result))
for item in result.loanStatusList:
    print(vars(item))

{'sid': 7, 'loanStatusList': [<__main__.LoanStatus object at 0x00000190A38CAE88>, <__main__.LoanStatus object at 0x00000190A2DBEAC8>, <__main__.LoanStatus object at 0x00000190A2DBEC88>, <__main__.LoanStatus object at 0x00000190A2DBEA08>], 'errorMessage': ''}
{'title': '데이터 분석 전문가 가이드  = The guide for advanced data analytics professional', 'RN': 'L0113396', 'CN': '005.74 데이한개', 'POS': '제1자료실(3층)', 'STATE': 0, 'RDD': '2021-03-01', 'BN': 1, 'errorMessage': ''}
{'title': '데이터 분석 전문가 가이드', 'RN': 'L0112135', 'CN': '005.74 한국데개  c.3', 'POS': '제1자료실(3층)', 'STATE': 0, 'RDD': '2020-04-02', 'BN': 0, 'errorMessage': ''}
{'title': '데이터 분석 전문가 가이드', 'RN': 'S0480361', 'CN': '005.74 한국데개', 'POS': '제1자료실(3층)', 'STATE': 1, 'RDD': '', 'BN': 0, 'errorMessage': ''}
{'title': '데이터 분석 전문가 가이드', 'RN': 'S0484218', 'CN': '005.74 한국데개  2', 'POS': '제1자료실(3층)', 'STATE': 0, 'RDD': '2020-02-17', 'BN': 0, 'errorMessage': ''}


In [ ]:
def makeJsonString(jsonDict):
    jsonString = ""

In [23]:
jsonDict = makejsonDict(result)

In [24]:
jsonDict

{'sid': 7,
 'loanStatusList': [{'title': '데이터 분석 전문가 가이드  = The guide for advanced data analytics professional',
   'RN': 'L0113396',
   'CN': '005.74 데이한개',
   'POS': '제1자료실(3층)',
   'STATE': 0,
   'RDD': '2021-03-01',
   'BN': 1,
   'errorMessage': ''},
  {'title': '데이터 분석 전문가 가이드',
   'RN': 'L0112135',
   'CN': '005.74 한국데개  c.3',
   'POS': '제1자료실(3층)',
   'STATE': 0,
   'RDD': '2020-04-02',
   'BN': 0,
   'errorMessage': ''},
  {'title': '데이터 분석 전문가 가이드',
   'RN': 'S0480361',
   'CN': '005.74 한국데개',
   'POS': '제1자료실(3층)',
   'STATE': 1,
   'RDD': '',
   'BN': 0,
   'errorMessage': ''},
  {'title': '데이터 분석 전문가 가이드',
   'RN': 'S0484218',
   'CN': '005.74 한국데개  2',
   'POS': '제1자료실(3층)',
   'STATE': 0,
   'RDD': '2020-02-17',
   'BN': 0,
   'errorMessage': ''}],
 'errorMessage': ''}

In [172]:
dictJson['sid'] = result.sid

In [4]:
ISBN = '9788988474839'
title = '데이터분석전문가가이드'
smlib_search_url = 'https://lib.smu.ac.kr/Search/?q=' 
#"https://lib.smu.ac.kr/Search/?q=%ec%82%ac%eb%9e%8c&searchTruncate=true&campuscode=00"
html = urlopen(smlib_search_url+ISBN)
bsObject = BeautifulSoup(html, "html.parser")

In [173]:
dictJson

{'sid': 7}

In [178]:
dictJson['loanStatusList'] = []

In [99]:
idx = 0
for link in bsObject.find_all('div', class_='search-list-result'):
  print(link)
  print(idx, "fuck")
  idx +=1

NameError: name 'bsObject' is not defined

In [179]:
for item in result.loanStatusList:
    dictJson['loanStatusList'].append(vars(item))

In [180]:
dictJson

{'sid': 7,
 'lodnStatusList': [],
 'loanStatusList': [{'title': '데이터 분석 전문가 가이드  = The guide for advanced data analytics professional',
   'RN': 'L0113396',
   'CN': '005.74 데이한개',
   'POS': '제1자료실(3층)',
   'STATE': 0,
   'RDD': '2021-03-01',
   'BN': 1,
   'errorMessage': ''},
  {'title': '데이터 분석 전문가 가이드',
   'RN': 'L0112135',
   'CN': '005.74 한국데개  c.3',
   'POS': '제1자료실(3층)',
   'STATE': 0,
   'RDD': '2020-04-02',
   'BN': 0,
   'errorMessage': ''},
  {'title': '데이터 분석 전문가 가이드',
   'RN': 'S0480361',
   'CN': '005.74 한국데개',
   'POS': '제1자료실(3층)',
   'STATE': 1,
   'RDD': '',
   'BN': 0,
   'errorMessage': ''},
  {'title': '데이터 분석 전문가 가이드',
   'RN': 'S0484218',
   'CN': '005.74 한국데개  2',
   'POS': '제1자료실(3층)',
   'STATE': 0,
   'RDD': '2020-02-17',
   'BN': 0,
   'errorMessage': ''}]}

In [181]:
dictJson['errorMessage']=result.errorMessage

In [175]:
result.loanStatusList

In [182]:
dictJson.pop('lodnStatusList')

[]

In [183]:
dictJson

{'sid': 7,
 'loanStatusList': [{'title': '데이터 분석 전문가 가이드  = The guide for advanced data analytics professional',
   'RN': 'L0113396',
   'CN': '005.74 데이한개',
   'POS': '제1자료실(3층)',
   'STATE': 0,
   'RDD': '2021-03-01',
   'BN': 1,
   'errorMessage': ''},
  {'title': '데이터 분석 전문가 가이드',
   'RN': 'L0112135',
   'CN': '005.74 한국데개  c.3',
   'POS': '제1자료실(3층)',
   'STATE': 0,
   'RDD': '2020-04-02',
   'BN': 0,
   'errorMessage': ''},
  {'title': '데이터 분석 전문가 가이드',
   'RN': 'S0480361',
   'CN': '005.74 한국데개',
   'POS': '제1자료실(3층)',
   'STATE': 1,
   'RDD': '',
   'BN': 0,
   'errorMessage': ''},
  {'title': '데이터 분석 전문가 가이드',
   'RN': 'S0484218',
   'CN': '005.74 한국데개  2',
   'POS': '제1자료실(3층)',
   'STATE': 0,
   'RDD': '2020-02-17',
   'BN': 0,
   'errorMessage': ''}],
 'errorMessage': ''}

In [135]:
html = urlopen('https://lib.smu.ac.kr/Search/Detail/819033?key=9788988474839&campuscode=00')
bsObject = BeautifulSoup(html, "html.parser")


In [147]:
includuedISBNString = str(bsObject.find('div', {'class':'col-md-10 detail-table-right'}).find_all('dl')[2].find('dd'))
find_ISBN(includuedISBNString)

In [148]:
a = find_ISBN(str(includuedISBNString))

In [149]:
a

'9788988474839'

In [129]:
#test
ISBN = '9788988474839'
sid = SM_CODE
title =  '데이터 분석 전문가 가이드  = The guide for advanced data analytics professional'
bookLink = '/Search/Detail/819033?key=9788988474839&campuscode=00'
test = visitLink(ISBN, sid, title, bookLink)

<td class="center-bu" scope="row">
                                            L0113396
                                                                                                                                </td>
------------------
<td class="center-bu leftsilla">005.74 데이한개   </td>
------------------
<td class="center-bu">
                                        제1자료실(3층)
                                    </td>
------------------
<td class="center-bu mobile_none">
<span class="sponge-textcolor-red">대출중</span><span class="sponge-textcolor-red">( 1명, 예약중)</span>
</td>
------------------
<td class="center-bu icon-mobile">
<span class="DeskTop_none"><span class="sponge-textcolor-red">대출중</span><span class="sponge-textcolor-red">( 1명, 예약중)</span></span><br class="DeskTop_none"/>
반납예정일 <span class="sponge-textcolor-red">2021-03-01 </span>
<a href="/account/Logon" title="예약"><span class="fa fa-chevron-circle-down"></span><span class="btn btn-xs btn-default ie7-visible">예약</span></

In [130]:
for item in test:
    print( vars(item))

{'title': '데이터 분석 전문가 가이드  = The guide for advanced data analytics professional', 'RN': 'L0113396', 'CN': '005.74 데이한개', 'POS': '제1자료실(3층)', 'STATE': 0, 'RDD': '2021-03-01', 'BN': 1, 'errorMessage': ''}


In [32]:
ISBN = '9788988474839'
title = '데이터 분석 전문가 가이드'
sid = SM_CODE
%debug(findLoanStatus(ISBN, title, sid))

UsageError: Line magic function `%debug(findLoanStatus(ISBN,` not found.


In [217]:
regexDictList = ['',
'',
'',
'',
'',
'',
'',
{'title': 'checkbox\d+\"\s+title=\"(?P<title>[^>]+)\"\stype=', 'link': '<a href=\"(?P<link>[^=]+key=[^>]+)\">'},
'',
'',
'',
'']

In [216]:
regexDictList[SM_CODE]['title']

'checkbox\\d+"\\s+title="(?P<title>[^=]+)"\\stype='

In [208]:
#Linklist 방문 테스트.
bookLinklist = [['데이터 분석 전문가 가이드  = The guide for advanced data analytics professional 선택', '/Search/Detail/819033?key=9788988474839&amp;campuscode=00'], ['데이터 분석 전문가 가이드 선택', '/Search/Detail/777677?key=9788988474839&amp;campuscode=00']]
for item in bookLinklist:
    visitLink(ISBN, sid, item[0], item[1])

/Search/Detail/819033?key=9788988474839&amp;campuscode=00
/Search/Detail/777677?key=9788988474839&amp;campuscode=00


In [82]:
asdf =visitLink('9788988474839', SM_CODE, '데이터 분석 전문가 가이드', 'Search/Detail/777677?key=9788988474839&campuscode=00')

<tr>
<td class="center-bu" scope="row">
                                            L0112135
                                                                                                                                </td>
<td class="center-bu leftsilla">005.74 한국데개  c.3 </td>
<td class="center-bu">
                                        제1자료실(3층)
                                    </td>
<td class="center-bu mobile_none">
<span class="sponge-textcolor-red">대출중</span>
</td>
<td class="center-bu icon-mobile">
<span class="DeskTop_none"><span class="sponge-textcolor-red">대출중</span></span><br class="DeskTop_none"/>
반납예정일 <span class="sponge-textcolor-red">2020-04-02 </span>
<a href="/account/Logon" title="예약"><span class="fa fa-chevron-circle-down"></span><span class="btn btn-xs btn-default ie7-visible">예약</span></a>
<a href="/Search/MyFolder/777677?campuscode=00" title="나의폴더"> <span class="fa fa-folder-open-o"></span><span class="btn btn-xs btn-default ie7-visible">마이폴더</span></a>
<

In [83]:
for item in asdf:
    print(vars(item))

{'title': '데이터 분석 전문가 가이드', 'RN': 'L0112135', 'CN': '005.74 한국데개  c.3', 'POS': '제1자료실(3층)', 'STATE': 0, 'RDD': '2020-04-02', 'BN': 0, 'errorMessage': ''}
{'title': '데이터 분석 전문가 가이드', 'RN': 'S0480361', 'CN': '005.74 한국데개', 'POS': '제1자료실(3층)', 'STATE': 1, 'RDD': '', 'BN': 0, 'errorMessage': ''}
{'title': '데이터 분석 전문가 가이드', 'RN': 'S0484218', 'CN': '005.74 한국데개  2', 'POS': '제1자료실(3층)', 'STATE': 0, 'RDD': '2020-02-17', 'BN': 0, 'errorMessage': ''}


In [42]:
"""test_string = ""
def crawling(ISBN, title, sid, searchUrl):
    libraryLoanStatus = LibraryLS(sid)
    try:
        #1. 접속.
        html = urlopen(searchUrl)
        bsObject = BeautifulSoup(html, "html.parser")
        #2. 링크를 차곡차곡저장.
        # 정규식을 이용하여 저장...!!
        bookLinklist =[]
        for bookResult in bsObject.find_all('div', class_='search-list-result'):
            bookResultString = str(bookResult)
            title = findSubStringByRegEx(bookResultString, sid, 'title')
            link = findSubStringByRegEx(bookResultString, sid, 'link')
            bookLinklist.append([title, link])
            print(title, link)
        print(bookLinklist)
        
        #3. 해당 링크를 방문하여라..!!
        #Efor item in bookLinklist
           visitLink(ISBN, sid, item[0], item[1])
    except AttributeError as e:
        traceback.print_exc()
        libraryLoanStatus.errorMessage = '해당 정규식에 포함되지 않는 예외케이스가 발생하였습니다. 정규식을 확인하고 수정하세요'
    finally:
        return libraryLoanStatus
    """

'test_string = ""\ndef crawling(ISBN, title, sid, searchUrl):\n    libraryLoanStatus = LibraryLS(sid)\n    try:\n        #1. 접속.\n        html = urlopen(searchUrl)\n        bsObject = BeautifulSoup(html, "html.parser")\n        #2. 링크를 차곡차곡저장.\n        # 정규식을 이용하여 저장...!!\n        bookLinklist =[]\n        for bookResult in bsObject.find_all(\'div\', class_=\'search-list-result\'):\n            bookResultString = str(bookResult)\n            title = findSubStringByRegEx(bookResultString, sid, \'title\')\n            link = findSubStringByRegEx(bookResultString, sid, \'link\')\n            bookLinklist.append([title, link])\n            print(title, link)\n        print(bookLinklist)\n        \n        #3. 해당 링크를 방문하여라..!!\n        #Efor item in bookLinklist\n           visitLink(ISBN, sid, item[0], item[1])\n    except AttributeError as e:\n        traceback.print_exc()\n        libraryLoanStatus.errorMessage = \'해당 정규식에 포함되지 않는 예외케이스가 발생하였습니다. 정규식을 확인하고 수정하세요\'\n    finally:\n        

In [239]:
"""def visitLink(ISBN, sid, title, bookLink):
    loanStatusList = []
    #= LoanStatus()
    try:
        #1. 접속.
        html = urlopen(base_start_url_list[sid]+bookLink)
        bsObject = BeautifulSoup(html, "html.parser")
        #2. 링크를 차곡차곡저장.
        # 정규식을 이용하여 저장...!!
        for loanResult in bsObject.find_all('div', class_='search-list-result'):
            loanResultString = str(loanResult)
            title = findSubStringByRegEx(bookResultString, sid, 'title')
            link = findSubStringByRegEx(bookResultString, sid, 'link')
            bookLinklist.append([title, link])
            print(title, link)
        print(bookLinklist)
        
        #3. 해당 링크를 방문하여라..!!
        #Efor item in bookLinklist
        #   visitLink(ISBN, sid, item[0], item[1])
    except AttributeError as e:
        traceback.print_exc()
        libraryLoanStatus.errorMessage = '해당 정규식에 포함되지 않는 예외케이스가 발생하였습니다. 정규식을 확인하고 수정하세요'
    finally:
        return libraryLoanStatus
"""

In [113]:
#에러 테스트 코드.
try:
    5/0
except BaseException as e:
    print(str(e))
    em = str(e)
    

division by zero


In [115]:
em

'division by zero'

In [137]:
titleRegex = re.compile('checkbox\d+\"\s+title=\"(?P<title>[^=]+)\"\stype=')
# (?P<title>[^=]+) 이거 앞뒤로 쪼개서 대학별로 붙여넣으면 딱일듯.
matchObj = titleRegex.search(text)
print(type(matchObj))
title = matchObj.group('title')

<class 're.Match'>


In [123]:
2/0

ZeroDivisionError: division by zero